<a href="https://colab.research.google.com/github/MovingJoon/Deeplearning-from-scratch/blob/master/Chapter_4%20%EC%8B%A0%EA%B2%BD%EB%A7%9D%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from mnist import load_mnist
from functions import softmax, cross_entropy_error, sigmoid
from gradient import numerical_gradient
#손실함수 (오차제곱합, 교차엔트로피)
def sum_squres_error(y,t):          #오차제곱합
    return 0.5* np.sum((y-t)**2)

def cross_entropy_error(y,t):       #교차 엔트로피
    if y.ndim == 1:                 #데이터가 1개인 행렬이라면
        t=t.reshape(1,t.size)       #1차원 배열로 전환
        y=y.reshape(1,y.size)       #1차원 배열로 전환
    batch_size= y.shape[0]          #데이터 갯수 60,000
    return -np.sum(t*np.log(y+1e-7))/batch_size

# 미분
def function(x):
    return x[0]**2+x[1]**2

# 기울기 구하기(편미분)
def numerical_diff(f,x):
    h=1e-4 #0.0001
    grad = np.zeros_like(x)

    for idx in range(x.size):
        temp=x[idx]     #x[0,1,2,3...]
        x[idx]=temp+h   #x+h
        result1=f(x)    #f(x+h)

        x[idx]=temp-h   #x-h
        result2=f(x)    #f(x-h)

        grad[idx]=(result1-result2)/(2*h) #f(x+h)-f(x-h)/2h
        x[idx]=temp                       #값 복원
    return grad                           #[x[0]에 대한 미분값, x[1]에 대한 미분값]

# gradient descent(경사하강법)
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x                            #초기값

    for i in range(step_num):
        grad=numerical_diff(f,x)          #초기값에대한 기울기
        x=x-lr*grad                       #[x0-(learning_rate*grad[0]),x1-(learning_rate*grad[1])]
    return x

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)     #가우스 정규분포로 초기화
    def predict(self,x):                  #입력값*가중치
        return np.dot(x,self.W)
    def loss(self,x,t):
        z=self.predict(x)                 #입력값*가중치
        y=softmax(z)                      #softmax(입력값*가중치)
        result=cross_entropy_error(y,t)   #교차 엔트로피를 손실함수로 사용
        return result




class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):

        self.params={}
        self.params['W1']=weight_init_std * np.random.randn(input_size,hidden_size)  #가중치 w1
        self.params['b1']=np.zeros(hidden_size)                                      #편향 b1
        self.params['W2']=weight_init_std * np.random.randn(hidden_size, output_size)#가중치 w2
        self.params['b2'] = np.zeros(output_size)                                    #편향 b2

    def predict(self,x):                                                             #순전파
        W1,W2=self.params['W1'],self.params['W2']
        b1,b2=self.params['b1'],self.params['b2']

        a1=np.dot(x,W1)+b1
        z1=sigmoid(a1)
        a2=np.dot(z1,W2)+b2
        y=softmax(a2)

        return y

    def loss(self, x,t):
        y=self.predict(x)                                                               #순전파
        return cross_entropy_error(y,t)                                                #손실함수 적용

    def accuracy(self,x,t):
        y=self.predict(x)                                                               #순전파
        y=np.argmax(y,axis=1)                                                          #가장큰값의 인덱스 출력(가로기준)
        t=np.argmax(t,axis=1)                                                          #가장큰값의 인덱스 출력(가로기준)

        accuracy = np.sum(y==t) / float(x.shape[0])                                    #일치된 정답의 수/전체데이터 수
        return accuracy

    def numerical_gradient(self,x,t):
        loss_W= lambda W: self.loss(x,t)                                               #손실함수 결과값 얻음
        grads={}
        grads['W1']=numerical_gradient(loss_W,self.params['W1'])                       #(손실함수 결과, 가중치값) 즉, 각 가중치값에 대하여 손실함수 값을 편미분한다.
        grads['b1']=numerical_gradient(loss_W,self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

In [ ]:
import numpy as np
from mnist import load_mnist
from test import TwoLayerNet

(x_train,t_train),(x_test,t_test)=load_mnist(normalize=True, one_hot_label=True)  #train 데이터와 test데이터분리

train_loss_list=[]

iters=10000                                                                       #반복횟수
train_size = x_train.shape[0]                                                     #데이터 크기
batch_size = 100                                                                  #배치사이즈
lr = 0.1                                                                          #learning rate

network= TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters):
    batch_mask=np.random.choice(train_size,batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grad = network.numerical_gradient(x_batch,t_batch)                            #기울기 계산

    for key in ('W1','b1','W2','b2'):
        network.params[key]=network.params[key] - lr*grad[key]                    #경사하강법적용

    result = network.loss(x_batch,t_batch)
    train_loss_list.append(result)                                                #반복문에서 i번재 마다의 손실함수 값 저장


print(train_loss_list.head(5))
